# Data Collecting

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import unicodedata

In [3]:
# Fetch the HTML content from the webpage
url = "https://books.toscrape.com/catalogue/page-"

# Initialize empty lists for book names and prices
books_names = []
books_prices = []
books_upcs = [] 
books_product_types = [] 
books_prices_excl_tax = []
books_prices_incl_tax = []
books_taxes = []
books_availabilities = []
books_num_reviews = []

# loop through all 10 pages
for page_num in range(1, 11):
    # Create the url for each page
    page_url = url+str(page_num)+ ".html"
    
    # Send a GET request to the page
    response = requests.get(page_url)
    
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all book's names
    for a_tag in soup.find_all('a', href=True):
        if 'title' in a_tag.attrs:
       
            #Extract book names from the a-tags
            book_name = a_tag['title'].strip()
            books_names.append(book_name)
            
    # Find all books' price
    for p_tag in soup.find_all('p', class_='price_color'):
        books_prices.append(p_tag.text)


In [4]:
# Define a function to clean the book name to match the weblink format
def clean_and_format(book_name):
    # Normalize characters and remove diacritic marks
    book_name_cleaned = ''.join(c for c in unicodedata.normalize('NFD', book_name) if unicodedata.category(c) != 'Mn')

    # Replace ampersand with a single hyphen
    book_name_cleaned = book_name_cleaned.replace('&', '-')

    # Replace non-alphanumeric characters with an empty string
    book_name_cleaned = re.sub(r'[^A-Za-z0-9\s\-\n]+', '', book_name_cleaned)

    # Check for numeric range in the book name
    numeric_range_match = re.search(r'([\d\s]+[+&:]+[\d\s]+)\s*(\D+\s*\d+)?', book_name_cleaned)

    if numeric_range_match:
        prefix, suffix = numeric_range_match.group(1), numeric_range_match.group(2)
        start_year, end_year = prefix.split('-')
        book_name_cleaned = book_name_cleaned.replace(prefix + '-' + suffix, f"{start_year}-{end_year}")

    # Replace spaces with hyphens and convert to lowercase
    book_name_cleaned = book_name_cleaned.replace(' ', '-').lower()

    # Replace consecutive hyphens with a single hyphen
    book_name_cleaned = re.sub(r'-+', '-', book_name_cleaned)

    return book_name_cleaned


# Loop through each book's name and index
for i, book_name in enumerate(books_names):
    book_name_cleaned = clean_and_format(book_name)

    # Calculate the index for the book
    book_index = 1000 - i

    # Create the URL for each book
    book_url = f"https://books.toscrape.com/catalogue/{book_name_cleaned}_{book_index}/index.html"

    # Call the function to scrape book info
    response = requests.get(book_url)
    if response.status_code == 200:
        book_soup = BeautifulSoup(response.text, 'html.parser')

        # Find the div containing product information
        product_info_div = book_soup.find('div', {'class': 'sub-header'})

        if product_info_div:
            # Find the table following the header
            product_info_table_header = product_info_div.find_next('h2', string='Product Information')

            if product_info_table_header:
                product_info_table = product_info_table_header.find_next('table')

                if product_info_table:
                    # Extract information from the table
                    rows = product_info_table.find_all('tr')
                    for row in rows:
                        columns = row.find_all(['th', 'td'])
                        column_name = columns[0].text.strip()
                        column_value = columns[1].text.strip()

                        # Append data to the respective lists
                        if column_name == 'UPC':
                            books_upcs.append(column_value)
                        elif column_name == 'Product Type':
                            books_product_types.append(column_value)
                        elif column_name == 'Price (excl. tax)':
                            books_prices_excl_tax.append(column_value)
                        elif column_name == 'Price (incl. tax)':
                            books_prices_incl_tax.append(column_value)
                        elif column_name == 'Tax':
                            books_taxes.append(column_value)
                        elif column_name == 'Availability':
                            books_availabilities.append(column_value)
                        elif column_name == 'Number of reviews':
                            books_num_reviews.append(column_value)

In [5]:
# Create a dataframe
data = pd.DataFrame({
                     'Title': books_names, 
                     'Book Price': books_prices,
                     'UPC':books_upcs,
                     'Product Type':books_product_types,
                     'Prices(excl. tax)':books_prices_excl_tax,
                     'Prices(incl. tax)':books_prices_incl_tax,
                     'Taxes':books_taxes,
                     'Availablities':books_availabilities,
                     'Number of review':books_num_reviews
                     })


In [6]:
data

,Title,Book Price,UPC,Product Type,Prices(excl. tax),Prices(incl. tax),Taxes,Availablities,Number of review
0,A Light in the Attic,£51.77,a897fe39b1053632,Books,Â£51.77,Â£51.77,Â£0.00,In stock (22 available),0
1,Tipping the Velvet,£53.74,90fa61229261140a,Books,Â£53.74,Â£53.74,Â£0.00,In stock (20 available),0
2,Soumission,£50.10,6957f44c3847a760,Books,Â£50.10,Â£50.10,Â£0.00,In stock (20 available),0
3,Sharp Objects,£47.82,e00eb4fd7b871a48,Books,Â£47.82,Â£47.82,Â£0.00,In stock (20 available),0
4,Sapiens: A Brief History of Humankind,£54.23,4165285e1663650f,Books,Â£54.23,Â£54.23,Â£0.00,In stock (20 available),0
...,...,...,...,...,...,...,...,...,...
195,Eureka Trivia 6.0,£54.59,29c0025455f8c585,Books,Â£54.59,Â£54.59,Â£0.00,In stock (15 available),0
196,Drive: The Surprising Truth About What Motivat...,£34.95,6ffb36aaeff1c81e,Books,Â£34.95,Â£34.95,Â£0.00,In stock (15 available),0
197,Done Rubbed Out (Reightman & Bailey #1),£37.72,f6d967cdadc6fbd9,Books,Â£37.72,Â£37.72,Â£0.00,In stock (15 available),0
198,Doing It Over (Most Likely To #1),£35.61,3a11bb962ff45b78,Books,Â£35.61,Â£35.61,Â£0.00,In stock (15 available),0


In [7]:
print(data)

                                                 Title Book Price  \
0                                 A Light in the Attic     £51.77   
1                                   Tipping the Velvet     £53.74   
2                                           Soumission     £50.10   
3                                        Sharp Objects     £47.82   
4                Sapiens: A Brief History of Humankind     £54.23   
..                                                 ...        ...   
195                                  Eureka Trivia 6.0     £54.59   
196  Drive: The Surprising Truth About What Motivat...     £34.95   
197            Done Rubbed Out (Reightman & Bailey #1)     £37.72   
198                  Doing It Over (Most Likely To #1)     £35.61   
199  Deliciously Ella Every Day: Quick and Easy Rec...     £42.16   

                  UPC Product Type Prices(excl. tax) Prices(incl. tax)  \
0    a897fe39b1053632        Books           Â£51.77           Â£51.77   
1    90fa61229261140a  

In [8]:
# Save the dataframe as a CSV file
data.to_csv('Project2_version5.csv')

# Data Processing

In [9]:
import pandas as pd

In [10]:
# Load the CSV file
data = pd.read_csv('Project2_version5.csv')

In [11]:
# Making 'Title' column values uppercase
data['Title'] = data['Title'].str.upper()

In [12]:
# Making 'Product Type' column values lowercase
data['Product Type'] = data['Product Type'].str.lower()


In [13]:
# Removing currency symbol from book price 
data['Book Price'] = data['Book Price'].str.replace('£', '')


In [14]:
# Extract numerical values (int) from 'Availability'
data['Availablities'] = data['Availablities'].str.extract('(\d+)').astype(float)


In [15]:
# Extract numerical values (int or float) from the 'Tax' column.

data['Taxes'] = data['Taxes'].apply(extract_numeric).astype(float)


NameError: name 'extract_numeric' is not defined

In [ ]:
# Extract numerical values (int or float) from these columns.
data['Prices(excl. tax)'] = data['Prices(excl. tax)'].apply(extract_numeric).astype(float)
data['Prices(incl. tax)'] = data['Prices(incl. tax)'].apply(extract_numeric).astype(float)


In [ ]:
# Save the processed dataframe to a new CSV file
data.to_csv('processed_data_version5.csv', index=False)

# Database

In [ ]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('books_database.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create a table in the database
cursor.execute('''
    CREATE TABLE IF NOT EXISTS books (
        id INTEGER PRIMARY KEY,
        title TEXT,
        book_price REAL,
        upc TEXT,
        product_type TEXT,
        price_excl_tax REAL,
        price_incl_tax REAL,
        tax REAL,
        availability REAL,
        num_reviews REAL
    )
''')

# Commit the changes to the database
conn.commit()

# Insert data into the table
for _, row in data.iterrows():
    cursor.execute('''
        INSERT INTO books (title, book_price, upc, product_type, price_excl_tax, price_incl_tax, tax, availability, num_reviews)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row['Title'], row['Book Price'], row['UPC'], row['Product Type'], row['Prices(excl. tax)'], row['Prices(incl. tax)'], row['Taxes'], row['Availablities'], row['Number of review']))

# Commit the changes to the database
conn.commit()

# Close the connection
conn.close()
print("Data has been successfully stored in the 'books_database.db' SQLite database.")

# Website

In [20]:
!pip install Flask Flask-SQLAlchemy

from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///C:\\Users\\john_\\College Study of Data Analysis\\DAB111_Intro to Python\\Project\\books_database.db"
db = SQLAlchemy(app)

In [21]:
# Define the Books model
class Book(db.Model):
    __tablename__ = 'books'
    upc = db.Column(db.String, primary_key=True)
    title = db.Column(db.String, nullable=False)
    book_price = db.Column(db.String, nullable=False)
    product_type = db.Column(db.String)
    price_excl_tax = db.Column(db.String)
    price_incl_tax = db.Column(db.String)
    tax = db.Column(db.String)
    availability = db.Column(db.String)
    num_reviews = db.Column(db.String)

In [22]:
@app.route("/")
def index():
    return render_template("index.html")

@app.route("/about")
def about():
    return render_template("about.html")

@app.route("/data_description")
def data_description():
    return render_template("data_description.html")  # Create this HTML template

@app.route("/search", methods=["GET", "POST"])
def search():
    if request.method == "POST":
        upc = request.form.get("upc")
        title = request.form.get("title")
        book_price = request.form.get("book_price")  # Add other form fields as needed

        if upc:
            books = Book.query.filter_by(upc=upc).first()
        elif title:
            books = Book.query.filter_by(title=title).first()
        elif book_price:
            books = Book.query.filter_by(book_price=book_price).first()
        # Add more conditions for other columns as needed
        else:
            books = None

        if books:
            return f"Book Found - Title: {books.title} | Price: {books.book_price}"
        else:
            flash("Record not found")
        return render_template("search_results.html", books=books)
    return render_template("search.html")  # Update this HTML template

@app.route("/update", methods=["GET", "POST"])
def update():
    if request.method == "POST":
        upc = request.form.get("upc")
        new_title = request.form.get("new_title")
        new_book_price = request.form.get("new_book_price")  # Add other form fields as needed

        if upc:
            book = Book.query.filter_by(upc=upc).first()
            if book:
                if new_title:
                    book.title = new_title
                if new_book_price:
                    book.book_price = new_book_price
                # Update other columns similarly
                db.session.commit()
                flash(f"Record updated successfully - UPC = {book.upc} ==> Title = {book.title}")
            else:
                flash("Record not found")
        return render_template("update_results.html")
    return render_template("update.html")  # Update this HTML template

@app.route("/delete", methods=["GET", "POST"])
def delete():
    if request.method == "POST":
        upc = request.form.get("book_id")
        book = Book.query.filter_by(upc=upc).first()
        if book:
            db.session.delete(book)
            db.session.commit()
            flash(f"Record deleted successfully - UPC = {book.upc} ==> Title = {book.title}")
        else:
            flash("Record not found")
        return render_template("delete_results.html")
    return render_template("delete.html")  # Update this HTML template

@app.route('/check_connection')
def check_connection():
    try:
        # Try to bind the session to the engine and execute a basic query
        db.session.bind = db.engine
        db.session.query("1").from_statement(db.text("SELECT 1")).all()
        return "Connected to the database!"
    except Exception as e:
        return str(e)


In [ ]:
if __name__ == "__main__":
    app.secret_key='admin123'
    app.run(debug=False)